In [2]:
import os
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct

URL = os.getenv("QDRANT_URL")
API_KEY = os.getenv("QDRANT_API_KEY")
COLLECTION_NAME = os.getenv("QDRANT_COLLECTION_NAME")

client = QdrantClient(url=URL, api_key=API_KEY)

# Add multiple keys in payload
points = [
    PointStruct(
        id=1,
        vector=[0.1, 0.2, 0.3],  # Example vector
        payload={
            "category": "news",
            "timestamp": "2024-12-30T10:00:00Z",
            "author": "Alice",
        },
    ),
    PointStruct(
        id=2,
        vector=[0.4, 0.5, 0.6],  # Example vector
        payload={
            "category": "blog",
            "timestamp": "2024-12-29T15:30:00Z",
            "author": "Bob",
        },
    ),
]

# Upsert points into Qdrant
client.upsert(collection_name="test", points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [5]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

# Create a filter to search by multiple keys
filter_query = Filter(
    must=[
        FieldCondition(key="category", match=MatchValue(value="news")),
        FieldCondition(key="author", match=MatchValue(value="Alice")),
    ]
)

# Search with the filter
result = client.search(
    collection_name="test",
    query_vector=[0.1, 0.2, 0.3],  # Query vector
)

In [6]:
from groq import Groq
import os

client = Groq(
    api_key="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",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
    stream=False,
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in the field of natural language processing (NLP) due to their ability to quickly and efficiently process and generate human-like language. The importance of fast language models can be seen in several areas:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation software. These models can quickly respond to user input, making the interaction more seamless and natural.
2. **Large-Scale Deployment**: Fast language models can be deployed on large-scale platforms, handling a high volume of requests and user interactions. This is particularly important for applications such as customer service chatbots, where speed and efficiency are critical.
3. **Improved User Experience**: Fast language models can significantly improve the user experience by reducing latency and providing quicker responses. This leads to increased user engagement, satisfaction, and loyalty.
4. **Ef